In [3]:
import pandas as pd
import getpass
import sqlalchemy as sa
from sqlalchemy import create_engine

driver = 'mysql+pymysql'
user = 'root'
password = getpass.getpass(prompt='Your password:'"")
ip = '127.0.0.1'
connection_string = f'{driver}://{user}:{password}@{ip}'

Your password:········


In [ ]:
db_connection = sa.create_engine(connection_string)
result = db_connection.execute('SELECT * FROM sakila.store')
for row in result:
    print(row)

In [ ]:
# 2. Write a Python function called rentals_month that retrieves rental data for a given month and year

def rentals_month(engine, month, year):
    query = "SELECT * from sakila.rental \
                WHERE date_format(rental_date, '%%m') = " + str(month) + \
                " AND date_format(rental_date, '%%Y') = " + str(year)
    rental = pd.read_sql_query(query, engine)
    return rental, month, year

df, m, y = rentals_month(db_connection, 5, 2005)
df
    
    

In [ ]:
# 3. Develop a Python function called rental_count_month that takes the DataFrame provided by rentals_month as input along with the month and year and returns a new DataFrame containing the number of rentals made by each customer_id during the selected month and year.

def rental_count_month(df, month, year):
    df = df.groupby('customer_id')[['rental_id']].count()
    df.rename(columns={'rental_id':f'rentals_{month}_{year}'}, inplace=True)
    return df

df1 = rental_count_month(df, m, y)
df1

In [ ]:
# 4. Create a Python function called compare_rentals that takes two DataFrames as input containing the number of rentals made by each customer in different months and years. The function should return a combined DataFrame with a new 'difference' column, which is the difference between the number of rentals in the two months.

def compare_rentals(df1, df2):
    df1.reset_index(inplace=True)
    df2.reset_index(inplace=True)
    df3 = df1.merge(df2, on='customer_id', how='outer')
    df3.fillna(0,inplace=True)
    df3['difference'] = df3[df3.columns[1]] - df3[df3.columns[2]]
    return df3

df1 = rental_count_month(*rentals_month(db_connection, 5, 2005))
df2 = rental_count_month(*rentals_month(db_connection, 6, 2005))
df3 = compare_rentals(df1, df2)
df3